![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# El `Hola mundo`de las redes neuronales con pyTorch

## Aprendizaje Automático Aplicado

### Maestría en Ciencia de Datos

#### **Julio Waissman**, 2024

[**Abrir en google Colab**](https://colab.research.google.com/github/mcd-unison/aaa-curso/blob/main/ejemplos/pytorch_ejemplo_simple.ipynb)


In [1]:
# Las librerías necesarias

import numpy as np
import matplotlib.pyplot as pl

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

## Cargando un conjunto de datos

El modulo `torchvision.datasets`contiene objetos `Dataset` para muchos conjuntos de datos de entrenamiento bien conocidos de imágenes como CIFAR, COCO etc.. (ver [documentación](https://pytorch.org/vision/stable/datasets.html)).

Vamos a usar `transform` para hacer un básico de procesamiento de las imágenes (tambien existe `transform_target`), así cada vez que cargue un conjunto de datos, los irá transformando.

In [21]:
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ]
)

# Descarga el conjunto de datos de entrenamiento
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Descarga el conjunto de datos de validación
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

Para poder usarlos en entrenamiento y prueba, hay que pasar los conjuntos de datos en forma de *mini batches*. Para eso usaremos `DataLoader`.

Es una función de wrap que permite, a partir de un `Dataset`, generar minibatches, muestrear, revolver aleatoriamente y cargar en diferentes hilos la carga de datos, para usarlos en entrenamiento o predicción.

In [22]:
BATCH_SIZE = 64

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)

test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

Y podemos ver como funciona, sacando el primer batch de test

In [ ]:
for X, y in test_dataloader:
    print("Ejemplo de un minibatch a ser usado")
    print(f"Shape de X [N, C, H, W]: {X.shape}")
    print(f"Shape de y: {y.shape} {y.dtype}")

    pl.figure(figsize=(8,8))
    for i in range(64):
      pl.subplot(8, 8, i+1)
      ax = pl.imshow(np.squeeze(X[i].numpy()), cmap='gray')
      pl.grid(visible=False)
      pl.axis('off')
      pl.title(y[i].numpy(), {'fontsize': 8, 'color': 'red'}, y = 0.91)
    break

## Funciones para el aprendizaje

Para poder realizar el aprendizaje, necesitamos implementar una función que nos permita hacer un epoch de optimización con los datos de entrenamiento, y otra función que nos permita verificar al final de cada epoch, si estamos aprendiendo o no, usando el conjunto de datos de validación.

Estas funciones sob bastante genéricas, aunque hay que programarlas (a diferencia de la función `compile` de TensorFlow):

In [25]:
def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 300 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Error en conjunto de test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Como puede verse, para hacer un paso de aprendizaje se necesita:

1. Un `DataLoader`que genere minibatches
2. Un modelo `model` de red neuronal en pytorch. Puede estar preentrenado o ser recien desarrollada su arquitectura.
3. Una función de pérdida `loss_fn`
4. Un método de optimización.

Empecemos por el modelo.

## Haciendo un modelo simple

Vamos a definir un modelo usando la forma de orientdo a objetos, que al parecer es por mucho la forma preferida en pyTorch, a diferencia de TensorFlow que privilegian la forma funcional.

La mayoría de las capas para redes neuronales se encuentran en el módulo [`torch.nn`](https://pytorch.org/docs/stable/nn.html) y las funciones que no requieren parámetros a aprender se encuentran en [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html#).

In [27]:
# Define el modelo en una combinación de orientado a objetos y funcional
class RedDensa(nn.Module):
    def __init__(self, hidden_units):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

y para poder entrenar se requiere establecer el modelo, asignarlo al dispositivo (CPU o GPU), definir la función de pérdida y el método de optimización:

In [ ]:
HIDDEN_UNITS = 128
EPOCHS = 10

# Get cpu, gpu or mps device for training.
device = (
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"Usando para el entrenamiento {device}")

modelo_simple = RedDensa(HIDDEN_UNITS).to(device)
print(modelo_simple)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelo_simple.parameters(), lr=1e-2)

## Entrenamiento

Y el entrenamiento se realiza entonces como:

In [ ]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n" + 36 * '-')

    train(train_dataloader, modelo_simple, loss_fn, optimizer)
    test(test_dataloader, modelo_simple, loss_fn)

print("¡Listo!")

## Guardando y cargando modelos

Vamos a ver como guardar solo los parámetros (asumiendo que podemos reconstruir un modelo similar) y luego como cargarlo a otro modelo para usarlos en reconocimiento.


In [ ]:
# Guardando solo los parámetros del modelo

torch.save(modelo_simple.state_dict(), "model.pth")

%ls

In [ ]:
#Cargando solo los parámetros del modelo, cuando conozco el modelo

model_loaded = RedDensa(HIDDEN_UNITS).to(device)
model_loaded.load_state_dict(torch.load("model.pth"))

In [ ]:
model_loaded.eval()

x = torch.stack([test_data[i][0] for i in range(10)])
y = test_data.targets[:10]

with torch.no_grad():
    x = x.to(device)
    pred = model_loaded(x)
    predicted = pred.argmax(dim=1)
    print(f'Predicted: "{predicted}", Actual: "{y}"')

## Una red convolucional tipo Le-Net

![alt_text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/convnet_fig.png "CNN Architecture for MNIST Classification")

In [33]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 24, 3, padding='valid')
        self.conv2 = nn.Conv2d(24, 36, 3, padding='valid')

        self.fc1 = nn.Linear(900, 128)
        self.fc2 = nn.Linear(128, 10)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = self.dropout1(x)

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))

        x = self.dropout2(x)
        logits = self.fc2(x)

        return logits


In [ ]:
# Los hiperparámetros
LR = 1e-4
EPOCHS = 5

device = (
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"Usando para el entrenamiento {device}")

# La definición del modelo
model_lenet = LeNet().to(device)
loss_fn_lenet = nn.CrossEntropyLoss()
optimizer_lenet = torch.optim.Adam(model_lenet.parameters(), lr=LR)

print(model_lenet)

In [ ]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n" + 36 * '-')
    train(train_dataloader, model_lenet, loss_fn_lenet, optimizer_lenet)
    test(test_dataloader, model_lenet, loss_fn_lenet)
print("¡Listo!")

## Para documentar nuestro optimismo

¿Podrías replicar el aprendizaje con otras arquitecturas y otros conjuntos de datos?

El reto es ahora hacer un modelo (y bajar los datos y usar un `DataLoader`) para dos conjuntos de datos interesantes:

1. [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)
2. [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)

Son dos conjuntos chiquitos y relativamente fáciles, que permiten probar con arquitecturas sencillas de CNNs.
